In [1]:
# importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

In [2]:
emp_df=pd.read_csv('processed_emp_data.csv')
emp_df.head()

,Age,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,JobSatisfaction,MaritalStatus,MonthlyIncome,NumCompaniesWorked,WorkLifeBalance,YearsAtCompany,Attrition
0,41,446,1,2,606,2,4,470,5993,8,1,6,1
1,49,961,8,1,606,3,2,673,5130,1,3,10,0
2,37,961,2,2,82,4,3,470,2090,6,3,0,1
3,33,961,3,4,606,4,3,673,2909,1,3,8,0
4,27,961,2,1,464,1,2,673,3468,9,3,2,0


In [3]:
# identifying X and y
X=emp_df.drop(['Attrition'],axis=1)
y=emp_df['Attrition']

In [4]:
# lets cross verfiy the result using train test split approach
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=21)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(1176, 12)
(294, 12)
(1176,)
(294,)


In [6]:
# Lets fit the model
from xgboost import XGBClassifier
xgb_model=XGBClassifier()

In [7]:
xgb_model.fit(X_train,y_train)

C:\Users\HP\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[12:44:36] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=8, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [9]:
# prediction on y
y_train_pred=xgb_model.predict(X_train)
y_test_pred=xgb_model.predict(X_test)

In [10]:
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
#accuracy check on train and test data
print('accuracy score of training data for xgb model: ',accuracy_score(y_train,y_train_pred))
print('accuracy score of testing data for xgb model: ',accuracy_score(y_test,y_test_pred))

accuracy score of training data for xgb model:  1.0
accuracy score of testing data for xgb model:  0.8197278911564626


In [11]:
# define model
model = XGBClassifier()
# define grid
weights = [1, 10, 25, 50, 75, 99, 100, 1000]
param_grid = dict(scale_pos_weight=weights)
# define evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# define grid search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring='roc_auc')
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

[12:47:48] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Best: 0.693246 using {'scale_pos_weight': 1}
0.693246 (0.046074) with: {'scale_pos_weight': 1}
0.686451 (0.044591) with: {'scale_pos_weight': 10}
0.677932 (0.040680) with: {'scale_pos_weight': 25}
0.677805 (0.044262) with: {'scale_pos_weight': 50}
0.676384 (0.052458) with: {'scale_pos_weight': 75}
0.675247 (0.045056) with: {'scale_pos_weight': 99}
0.676697 (0.048116) with: {'scale_pos_weight': 100}
0.665723 (0.055153) with: {'scale_pos_weight': 1000}


C:\Users\HP\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
